# Unified Electrophysiology Analysis Pipeline

This notebook processes ABF files to detect spikes and bursts, classify bursts, extract normalized bursts for shapelet learning, train a shapelet model, perform UMAP embedding, and detect conflict regions between burst types.

All sections include comments outside the code explaining their purpose.

In [ ]:
# Imports and parameters
import os
import numpy as np
import pandas as pd
import pyabf
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.signal import find_peaks, hilbert
from scipy.stats import skew, kurtosis
from scipy.interpolate import interp1d
from numpy.linalg import lstsq
from sklearn.preprocessing import StandardScaler
import umap
from sklearn.neighbors import NearestNeighbors
from tslearn.preprocessing import TimeSeriesScalerMinMax
from tslearn.shapelets import LearningShapelets
from tensorflow.keras.optimizers import Adam
from matplotlib import cm

# Parameters
folder_path = "bursting"
threshold = -35
burst_threshold = 0.3
fs = 10000
dt = 1/fs

In [ ]:
# Helper functions for spikes and bursts
def normalize_y(signal_segment):
    return (signal_segment - np.mean(signal_segment)) / np.std(signal_segment)

def rescale_x(time_segment, signal_segment, n_points=100):
    f = interp1d(np.linspace(0, 1, len(signal_segment)), signal_segment)
    return f(np.linspace(0, 1, n_points))

def detect_spikes(signal, threshold=-35):
    spike_indices, _ = find_peaks(signal, height=threshold)
    return spike_indices

def detect_bursts(spike_times, burst_threshold=0.3):
    isi = np.diff(spike_times)
    bursts = []
    current_burst = [spike_times[0]]
    for i in range(1, len(isi)):
        if isi[i-1] < burst_threshold:
            current_burst.append(spike_times[i])
        else:
            if len(current_burst) > 1:
                bursts.append((current_burst[0], current_burst[-1]))
            current_burst = [spike_times[i]]
    if len(current_burst) > 1:
        bursts.append((current_burst[0], current_burst[-1]))
    return bursts

In [ ]:
# Burst classification
def classify_bursts(bursts, signal, time):
    square_wave_bursts = []
    parabolic_bursts = []
    other_bursts = []

    for i, (burst_start, burst_end) in enumerate(bursts):
        burst_mask = (time >= burst_start) & (time <= burst_end)
        burst_min = np.min(signal[burst_mask])

        prev_mean = np.mean(signal[(time > bursts[i-1][1]) & (time < burst_start)]) if i>0 else np.nan
        next_mean = np.mean(signal[(time > burst_end) & (time < bursts[i+1][0])]) if i < len(bursts)-1 else np.nan
        inter_mean = np.nanmean([prev_mean, next_mean])

        if burst_min > inter_mean:
            square_wave_bursts.append((burst_start, burst_end))
        elif burst_min < inter_mean:
            parabolic_bursts.append((burst_start, burst_end))
        else:
            other_bursts.append((burst_start, burst_end))
    return square_wave_bursts, parabolic_bursts, other_bursts

In [ ]:
# Extract normalized bursts for shapelet learning
def extract_normalized_bursts(burst_list, signal, time, n_points=100):
    normalized_bursts = []
    for start, end in burst_list:
        mask = (time >= start) & (time <= end)
        s_rescaled = rescale_x(time[mask], signal[mask], n_points)
        s_normalized = normalize_y(s_rescaled)
        normalized_bursts.append(s_normalized)
    return normalized_bursts